Running Your Queries In Spark
---------------------------

You need to take the data from Twitter and perform your analysis based on the question you chose.

In our hashtag counting example, we do the following:
1. Explode the tweet into its individual words.
2. Filter out hashtags, then each tag gets assigned the count 1 with the `map` method.
3. Append the current hashtags to the main dataframe and update the counts appropriately.
4. Query the temporary table to return the current top 10 hashtags trending, and show to the console.

You can extend this into a web dashboard, or plots inside this notebook if you choose.

----------------------------------------------------------------------------------------------------------------------

#### Import the relevant modules

In [1]:
from pyspark import SparkConf,SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import Row,SQLContext, SparkSession
import sys
import requests

#### Set some constants, initialize Spark, and then open the socket with the remote host.

In [2]:
TCP_REMOTE_HOST = "data_server"
TCP_PORT = 9009

# create spark configuration
conf = SparkConf()
conf.setAppName("TwitterStreamApp")

# create spark context with the above configuration
sc = SparkContext(conf=conf)
sc.setLogLevel("ERROR")

# create the Streaming Context from the above spark context with interval size 2 seconds
ssc = StreamingContext(sc, 2)

# setting a checkpoint to allow RDD recovery
ssc.checkpoint("checkpoint_TwitterApp")

# read data from port 9009
dataStream = ssc.socketTextStream(TCP_REMOTE_HOST, TCP_PORT)

#### We now create some helper functions to allow Spark to maintain our running count.

In [3]:
def aggregate_tags_count(new_values, total_sum):
    return sum(new_values) + (total_sum or 0)

def getSparkSessionInstance(sparkConf):
    if ("sparkSessionSingletonInstance" not in globals()):
        globals()["sparkSessionSingletonInstance"] = SparkSession \
            .builder \
            .config(conf=sparkConf) \
            .getOrCreate()
    return globals()["sparkSessionSingletonInstance"]

def process_rdd(time, rdd):
    print(f"----------- {str(time)} -----------")
    try:
        # Get spark sql singleton context from the current context
        spark = getSparkSessionInstance(rdd.context.getConf())
        
        # convert the RDD to Row RDD
        row_rdd = rdd.map(lambda w: Row(hashtag=w[0], hashtag_count=w[1]))
        
        # create a DF from the Row RDD
        hashtags_df = spark.createDataFrame(row_rdd)
        
        # Register the dataframe as table
        hashtags_df.registerTempTable("hashtags")
        #hashtags_df.createOrReplaceTempView("hashtags")
        
        # get the top 10 hashtags from the table using SQL and print them
        hashtag_counts_df = spark.sql(
            "select hashtag, hashtag_count from hashtags order by hashtag_count desc limit 10")
        hashtag_counts_df.show()
        
        total_counts_df = spark.sql(
           "select sum(hashtag_count) from hashtags")
        total_counts_df.show()

    except Exception as e:
        e = sys.exc_info()[1]
        print(f"Error: {e}")

#### Finally, we assign our primary workflow that will utilize the above functions.

After that's complete, we begin the streaming with `ssc.start()`. The query stays open until we terminate it (`ssc.awaitTermination()`).

We can end the streaming by going to the `data_server` container and typing `ctrl-C`. Recall that you can exit the container shell with the `Ctrl-P, Ctrl-Q` sequence.

If you are using tweets from a geofenced area (as we are here) - you should let it run for a while in order to build up enough data to be useful.

sc = SparkContext.getOrCreate()
ssc = StreamingContext(sc, 5)
sqlContext = SQLContext(sc)

def streamrdd_to_df(srdd):
    sdf = sqlContext.createDataFrame(srdd)
    sdf.show(n=2, truncate=False)
    return sdf

In [4]:
from operator import add

# split each tweet into words
words = dataStream.flatMap(lambda line: line.split(" "))

table_df = words.map(lambda word: (word, 1)).reduceByKey(add)

# filter the words to get only hashtags, then map each hashtag to be a pair of (hashtag,1)
hashtags = words.filter(lambda w: '#' in w).map(lambda x: (x, 1))

# adding the count of each hashtag to its last count
tags_totals = hashtags.updateStateByKey(aggregate_tags_count)

# do processing for each RDD generated in each interval
tags_totals.foreachRDD(process_rdd)

# start the streaming computation
ssc.start()

# wait for the streaming to finish
ssc.awaitTermination()

----------- 2019-02-05 20:15:00 -----------
Error: RDD is empty
----------- 2019-02-05 20:15:02 -----------
Error: RDD is empty
----------- 2019-02-05 20:15:04 -----------
Error: RDD is empty
----------- 2019-02-05 20:15:06 -----------
Error: RDD is empty
----------- 2019-02-05 20:15:08 -----------
Error: RDD is empty
----------- 2019-02-05 20:15:10 -----------
Error: RDD is empty
----------- 2019-02-05 20:15:12 -----------
Error: RDD is empty
----------- 2019-02-05 20:15:14 -----------
Error: RDD is empty
----------- 2019-02-05 20:15:16 -----------
Error: RDD is empty
----------- 2019-02-05 20:15:18 -----------
Error: RDD is empty
----------- 2019-02-05 20:15:20 -----------
Error: RDD is empty
----------- 2019-02-05 20:15:22 -----------
+-----------------+-------------+
|          hashtag|hashtag_count|
+-----------------+-------------+
|       #milkshake|            1|
|         #dessert|            1|
|    #strategizing|            1|
|            #food|            1|
|    #alexxant

----------- 2019-02-05 20:15:46 -----------
+-------------------+-------------+
|            hashtag|hashtag_count|
+-------------------+-------------+
|          #carolina|            2|
|             #BWOOD|            2|
|            #SINNER|            2|
|         #madeinnyc|            2|
|       #GOONEYSAINT|            2|
|               #Sam|            1|
|disappearing?#SAINT|            1|
|           #dessert|            1|
|     #TradesTalkThe|            1|
|               #pug|            1|
+-------------------+-------------+

+------------------+
|sum(hashtag_count)|
+------------------+
|                32|
+------------------+

----------- 2019-02-05 20:15:48 -----------
+-------------+-------------+
|      hashtag|hashtag_count|
+-------------+-------------+
|    #carolina|            2|
| #GOONEYSAINT|            2|
|      #SINNER|            2|
|   #madeinnyc|            2|
|       #BWOOD|            2|
|   #milkshake|            1|
|#strategizing|            1|
|

----------- 2019-02-05 20:16:12 -----------
+-------------------+-------------+
|            hashtag|hashtag_count|
+-------------------+-------------+
|            #SINNER|            2|
|             #BWOOD|            2|
|         #madeinnyc|            2|
|          #carolina|            2|
|       #GOONEYSAINT|            2|
|              #I678|            1|
|disappearing?#SAINT|            1|
|           #dessert|            1|
|     #TradesTalkThe|            1|
|               #Sam|            1|
+-------------------+-------------+

+------------------+
|sum(hashtag_count)|
+------------------+
|                42|
+------------------+

----------- 2019-02-05 20:16:14 -----------
+--------------------+-------------+
|             hashtag|hashtag_count|
+--------------------+-------------+
|              #BWOOD|            2|
|           #carolina|            2|
|          #madeinnyc|            2|
|             #SINNER|            2|
|        #GOONEYSAINT|            2|
|💐💐💐🌷

----------- 2019-02-05 20:16:38 -----------
+--------------------+-------------+
|             hashtag|hashtag_count|
+--------------------+-------------+
|          #madeinnyc|            2|
|        #GOONEYSAINT|            2|
|             #SINNER|            2|
|              #BWOOD|            2|
|           #carolina|            2|
|        #Litchfield,|            1|
|#Cosmetology@Dear...|            1|
|               #food|            1|
|       #alexxantaeus|            1|
|              #class|            1|
+--------------------+-------------+

+------------------+
|sum(hashtag_count)|
+------------------+
|                48|
+------------------+

----------- 2019-02-05 20:16:40 -----------
+-------------------+-------------+
|            hashtag|hashtag_count|
+-------------------+-------------+
|          #carolina|            2|
|       #GOONEYSAINT|            2|
|         #madeinnyc|            2|
|            #SINNER|            2|
|             #BWOOD|            2|

----------- 2019-02-05 20:17:04 -----------
+-------------------+-------------+
|            hashtag|hashtag_count|
+-------------------+-------------+
|             #BWOOD|            2|
|            #SINNER|            2|
|         #madeinnyc|            2|
|          #carolina|            2|
|       #GOONEYSAINT|            2|
|               #Sam|            1|
|disappearing?#SAINT|            1|
|           #dessert|            1|
|     #TradesTalkThe|            1|
|        #timelapse…|            1|
+-------------------+-------------+

+------------------+
|sum(hashtag_count)|
+------------------+
|                55|
+------------------+

----------- 2019-02-05 20:17:06 -----------
+-------------------+-------------+
|            hashtag|hashtag_count|
+-------------------+-------------+
|       #GOONEYSAINT|            2|
|            #SINNER|            2|
|         #madeinnyc|            2|
|             #BWOOD|            2|
|          #carolina|            2|
|        #cra

...

----------- 2019-02-05 20:22:42 -----------
+-------------------+-------------+
|            hashtag|hashtag_count|
+-------------------+-------------+
|              #I278|            3|
|       #GOONEYSAINT|            2|
|         #madeinnyc|            2|
|             #BWOOD|            2|
|            #SINNER|            2|
|          #carolina|            2|
|disappearing?#SAINT|            1|
|           #dessert|            1|
|     #TradesTalkThe|            1|
|        #timelapse…|            1|
+-------------------+-------------+

+------------------+
|sum(hashtag_count)|
+------------------+
|                81|
+------------------+

----------- 2019-02-05 20:22:44 -----------
+-------------------+-------------+
|            hashtag|hashtag_count|
+-------------------+-------------+
|              #I278|            3|
|       #GOONEYSAINT|            2|
|          #carolina|            2|
|            #SINNER|            2|
|         #madeinnyc|            2|
|            

----------- 2019-02-05 20:23:08 -----------
+------------+-------------+
|     hashtag|hashtag_count|
+------------+-------------+
|       #I278|            3|
|  #madeinnyc|            2|
|     #SINNER|            2|
|#GOONEYSAINT|            2|
|      #BWOOD|            2|
|   #carolina|            2|
| #Manhasset,|            1|
|   #NYGiants|            1|
|  #ElClásico|            1|
|  #pipfamily|            1|
+------------+-------------+

+------------------+
|sum(hashtag_count)|
+------------------+
|                81|
+------------------+

----------- 2019-02-05 20:23:10 -----------
+-------------------+-------------+
|            hashtag|hashtag_count|
+-------------------+-------------+
|              #I278|            3|
|          #carolina|            2|
|             #BWOOD|            2|
|            #SINNER|            2|
|         #madeinnyc|            2|
|       #GOONEYSAINT|            2|
|disappearing?#SAINT|            1|
|           #dessert|            1|
|  

----------- 2019-02-05 20:23:34 -----------
+-------------------+-------------+
|            hashtag|hashtag_count|
+-------------------+-------------+
|              #I278|            3|
|          #carolina|            2|
|         #madeinnyc|            2|
|             #BWOOD|            2|
|            #SINNER|            2|
|       #GOONEYSAINT|            2|
|disappearing?#SAINT|            1|
|           #dessert|            1|
|     #TradesTalkThe|            1|
|        #timelapse…|            1|
+-------------------+-------------+

+------------------+
|sum(hashtag_count)|
+------------------+
|                81|
+------------------+

----------- 2019-02-05 20:23:36 -----------
+-------------------+-------------+
|            hashtag|hashtag_count|
+-------------------+-------------+
|              #I278|            3|
|            #SINNER|            2|
|          #carolina|            2|
|       #GOONEYSAINT|            2|
|         #madeinnyc|            2|
|            

KeyboardInterrupt: 

----------- 2019-02-05 20:23:52 -----------
+-------------------+-------------+
|            hashtag|hashtag_count|
+-------------------+-------------+
|              #I278|            3|
|          #carolina|            2|
|         #madeinnyc|            2|
|            #SINNER|            2|
|             #BWOOD|            2|
|       #GOONEYSAINT|            2|
|disappearing?#SAINT|            1|
|           #dessert|            1|
|     #TradesTalkThe|            1|
|        #timelapse…|            1|
+-------------------+-------------+

+------------------+
|sum(hashtag_count)|
+------------------+
|                81|
+------------------+

----------- 2019-02-05 20:23:54 -----------
+-------------------+-------------+
|            hashtag|hashtag_count|
+-------------------+-------------+
|              #I278|            3|
|          #carolina|            2|
|       #GOONEYSAINT|            2|
|             #BWOOD|            2|
|         #madeinnyc|            2|
|            

In [6]:
testing = words.map(lambda word: (word, 1)).reduceByKey(add)


----------- 2019-02-05 20:34:00 -----------
+-------------------+-------------+
|            hashtag|hashtag_count|
+-------------------+-------------+
|              #I278|            3|
|       #GOONEYSAINT|            2|
|            #SINNER|            2|
|             #BWOOD|            2|
|         #madeinnyc|            2|
|          #carolina|            2|
|disappearing?#SAINT|            1|
|           #dessert|            1|
|     #TradesTalkThe|            1|
|        #timelapse…|            1|
+-------------------+-------------+

+------------------+
|sum(hashtag_count)|
+------------------+
|                81|
+------------------+

----------- 2019-02-05 20:34:02 -----------
+-------------------+-------------+
|            hashtag|hashtag_count|
+-------------------+-------------+
|              #I278|            3|
|          #carolina|            2|
|         #madeinnyc|            2|
|       #GOONEYSAINT|            2|
|             #BWOOD|            2|
|            

In [7]:
testing.show()

AttributeError: 'TransformedDStream' object has no attribute 'show'

----------- 2019-02-05 20:34:14 -----------
+-------------------+-------------+
|            hashtag|hashtag_count|
+-------------------+-------------+
|              #I278|            3|
|       #GOONEYSAINT|            2|
|         #madeinnyc|            2|
|            #SINNER|            2|
|          #carolina|            2|
|             #BWOOD|            2|
|disappearing?#SAINT|            1|
|           #dessert|            1|
|     #TradesTalkThe|            1|
|        #timelapse…|            1|
+-------------------+-------------+

+------------------+
|sum(hashtag_count)|
+------------------+
|                81|
+------------------+

----------- 2019-02-05 20:34:16 -----------
+-------------------+-------------+
|            hashtag|hashtag_count|
+-------------------+-------------+
|              #I278|            3|
|          #carolina|            2|
|         #madeinnyc|            2|
|       #GOONEYSAINT|            2|
|            #SINNER|            2|
|            

In [10]:
streamrdd_to_df(words)

TypeError: 'TransformedDStream' object is not iterable

----------- 2019-02-05 20:37:34 -----------
+-------------------+-------------+
|            hashtag|hashtag_count|
+-------------------+-------------+
|              #I278|            3|
|            #SINNER|            2|
|         #madeinnyc|            2|
|          #carolina|            2|
|             #BWOOD|            2|
|       #GOONEYSAINT|            2|
|disappearing?#SAINT|            1|
|           #dessert|            1|
|     #TradesTalkThe|            1|
|        #timelapse…|            1|
+-------------------+-------------+

+------------------+
|sum(hashtag_count)|
+------------------+
|                81|
+------------------+

----------- 2019-02-05 20:37:36 -----------
+-------------------+-------------+
|            hashtag|hashtag_count|
+-------------------+-------------+
|              #I278|            3|
|          #carolina|            2|
|         #madeinnyc|            2|
|            #SINNER|            2|
|       #GOONEYSAINT|            2|
|            

----------- 2019-02-05 20:38:00 -----------
+-------------------+-------------+
|            hashtag|hashtag_count|
+-------------------+-------------+
|              #I278|            3|
|          #carolina|            2|
|             #BWOOD|            2|
|            #SINNER|            2|
|         #madeinnyc|            2|
|       #GOONEYSAINT|            2|
|disappearing?#SAINT|            1|
|           #dessert|            1|
|     #TradesTalkThe|            1|
|        #timelapse…|            1|
+-------------------+-------------+

+------------------+
|sum(hashtag_count)|
+------------------+
|                81|
+------------------+

----------- 2019-02-05 20:38:02 -----------
+-------------------+-------------+
|            hashtag|hashtag_count|
+-------------------+-------------+
|              #I278|            3|
|       #GOONEYSAINT|            2|
|            #SINNER|            2|
|          #carolina|            2|
|         #madeinnyc|            2|
|            

----------- 2019-02-05 20:38:26 -----------
+-------------------+-------------+
|            hashtag|hashtag_count|
+-------------------+-------------+
|              #I278|            3|
|            #SINNER|            2|
|         #madeinnyc|            2|
|          #carolina|            2|
|       #GOONEYSAINT|            2|
|             #BWOOD|            2|
|disappearing?#SAINT|            1|
|           #dessert|            1|
|     #TradesTalkThe|            1|
|        #timelapse…|            1|
+-------------------+-------------+

+------------------+
|sum(hashtag_count)|
+------------------+
|                81|
+------------------+

----------- 2019-02-05 20:38:28 -----------
+------------+-------------+
|     hashtag|hashtag_count|
+------------+-------------+
|       #I278|            3|
|   #carolina|            2|
|     #SINNER|            2|
|      #BWOOD|            2|
|#GOONEYSAINT|            2|
|  #madeinnyc|            2|
| #Manhasset,|            1|
|   #NYGian

----------- 2019-02-05 20:38:52 -----------
+-------------------+-------------+
|            hashtag|hashtag_count|
+-------------------+-------------+
|              #I278|            3|
|       #GOONEYSAINT|            2|
|         #madeinnyc|            2|
|          #carolina|            2|
|            #SINNER|            2|
|             #BWOOD|            2|
|disappearing?#SAINT|            1|
|           #dessert|            1|
|     #TradesTalkThe|            1|
|        #timelapse…|            1|
+-------------------+-------------+

+------------------+
|sum(hashtag_count)|
+------------------+
|                81|
+------------------+

----------- 2019-02-05 20:38:54 -----------
+-------------------+-------------+
|            hashtag|hashtag_count|
+-------------------+-------------+
|              #I278|            3|
|       #GOONEYSAINT|            2|
|             #BWOOD|            2|
|          #carolina|            2|
|         #madeinnyc|            2|
|            

...

----------- 2019-02-05 20:54:02 -----------
+-------------------+-------------+
|            hashtag|hashtag_count|
+-------------------+-------------+
|              #I278|            3|
|       #GOONEYSAINT|            2|
|         #madeinnyc|            2|
|            #SINNER|            2|
|             #BWOOD|            2|
|          #carolina|            2|
|disappearing?#SAINT|            1|
|           #dessert|            1|
|     #TradesTalkThe|            1|
|        #timelapse…|            1|
+-------------------+-------------+

+------------------+
|sum(hashtag_count)|
+------------------+
|                81|
+------------------+

----------- 2019-02-05 20:54:04 -----------
+-------------------+-------------+
|            hashtag|hashtag_count|
+-------------------+-------------+
|              #I278|            3|
|          #carolina|            2|
|             #BWOOD|            2|
|       #GOONEYSAINT|            2|
|         #madeinnyc|            2|
|            

----------- 2019-02-05 20:54:28 -----------
+--------------------+-------------+
|             hashtag|hashtag_count|
+--------------------+-------------+
|               #I278|            3|
|        #GOONEYSAINT|            2|
|          #madeinnyc|            2|
|              #BWOOD|            2|
|           #carolina|            2|
|             #SINNER|            2|
|            #winning|            1|
|#brothafromanotha...|            1|
|#ReasonsYouWereMu...|            1|
|        #photography|            1|
+--------------------+-------------+

+------------------+
|sum(hashtag_count)|
+------------------+
|                81|
+------------------+

----------- 2019-02-05 20:54:30 -----------
+-------------------+-------------+
|            hashtag|hashtag_count|
+-------------------+-------------+
|              #I278|            3|
|          #carolina|            2|
|            #SINNER|            2|
|       #GOONEYSAINT|            2|
|         #madeinnyc|            2|

----------- 2019-02-05 20:54:54 -----------
+-------------------+-------------+
|            hashtag|hashtag_count|
+-------------------+-------------+
|              #I278|            3|
|          #carolina|            2|
|         #madeinnyc|            2|
|             #BWOOD|            2|
|       #GOONEYSAINT|            2|
|            #SINNER|            2|
|disappearing?#SAINT|            1|
|           #dessert|            1|
|     #TradesTalkThe|            1|
|        #timelapse…|            1|
+-------------------+-------------+

+------------------+
|sum(hashtag_count)|
+------------------+
|                81|
+------------------+

----------- 2019-02-05 20:54:56 -----------
+-------------------+-------------+
|            hashtag|hashtag_count|
+-------------------+-------------+
|              #I278|            3|
|       #GOONEYSAINT|            2|
|            #SINNER|            2|
|             #BWOOD|            2|
|         #madeinnyc|            2|
|          #c

----------- 2019-02-05 20:55:20 -----------
+-------------------+-------------+
|            hashtag|hashtag_count|
+-------------------+-------------+
|              #I278|            3|
|             #BWOOD|            2|
|         #madeinnyc|            2|
|          #carolina|            2|
|       #GOONEYSAINT|            2|
|            #SINNER|            2|
|disappearing?#SAINT|            1|
|           #dessert|            1|
|     #TradesTalkThe|            1|
|        #timelapse…|            1|
+-------------------+-------------+

+------------------+
|sum(hashtag_count)|
+------------------+
|                81|
+------------------+

----------- 2019-02-05 20:55:22 -----------
+-------------------+-------------+
|            hashtag|hashtag_count|
+-------------------+-------------+
|              #I278|            3|
|          #carolina|            2|
|         #madeinnyc|            2|
|             #BWOOD|            2|
|            #SINNER|            2|
|       #GOON

----------- 2019-02-05 20:55:46 -----------
+-------------------+-------------+
|            hashtag|hashtag_count|
+-------------------+-------------+
|              #I278|            3|
|       #GOONEYSAINT|            2|
|          #carolina|            2|
|             #BWOOD|            2|
|         #madeinnyc|            2|
|            #SINNER|            2|
|disappearing?#SAINT|            1|
|           #dessert|            1|
|     #TradesTalkThe|            1|
|        #timelapse…|            1|
+-------------------+-------------+

+------------------+
|sum(hashtag_count)|
+------------------+
|                81|
+------------------+

----------- 2019-02-05 20:55:48 -----------
+-------------------+-------------+
|            hashtag|hashtag_count|
+-------------------+-------------+
|              #I278|            3|
|       #GOONEYSAINT|            2|
|         #madeinnyc|            2|
|             #BWOOD|            2|
|          #carolina|            2|
|            

In [11]:
scc.stop()

NameError: name 'scc' is not defined

+-------------------+-------------+
|            hashtag|hashtag_count|
+-------------------+-------------+
|              #I278|            3|
|          #carolina|            2|
|         #madeinnyc|            2|
|             #BWOOD|            2|
|            #SINNER|            2|
|       #GOONEYSAINT|            2|
|disappearing?#SAINT|            1|
|           #dessert|            1|
|     #TradesTalkThe|            1|
|        #timelapse…|            1|
+-------------------+-------------+

+------------------+
|sum(hashtag_count)|
+------------------+
|                81|
+------------------+

----------- 2019-02-05 20:55:54 -----------
+-------------------+-------------+
|            hashtag|hashtag_count|
+-------------------+-------------+
|              #I278|            3|
|            #SINNER|            2|
|         #madeinnyc|            2|
|          #carolina|            2|
|             #BWOOD|            2|
|       #GOONEYSAINT|            2|
|disappearing?#SAINT|

In [12]:
ssc.stop()

+-------------------+-------------+
|            hashtag|hashtag_count|
+-------------------+-------------+
|              #I278|            3|
|          #carolina|            2|
|             #BWOOD|            2|
|       #GOONEYSAINT|            2|
|         #madeinnyc|            2|
|            #SINNER|            2|
|disappearing?#SAINT|            1|
|           #dessert|            1|
|     #TradesTalkThe|            1|
|        #timelapse…|            1|
+-------------------+-------------+

Error: An error occurred while calling o192298.showString.
: java.lang.IllegalStateException: SparkContext has been shutdown
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2021)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2050)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2069)
	at org.apache.spark.sql.execution.SparkPlan.executeTake(SparkPlan.scala:336)
	at org.apache.spark.sql.execution.CollectLimitExec.executeCollect(limit.scala:38)
	at org.apa

In [18]:
countByValue(testing)

NameError: name 'countByValue' is not defined

In [21]:
from pyspark.streaming import countByValue

ImportError: cannot import name 'countByValue'

In [23]:
testing.countByValue()

AttributeError: 'NoneType' object has no attribute 'sc'

In [31]:
print(hashtags)

In [25]:
pprint(testing)

Pretty printing has been turned OFF


In [29]:
ssc.count(testing)

AttributeError: 'StreamingContext' object has no attribute 'count'

In [30]:
words.collect()

AttributeError: 'TransformedDStream' object has no attribute 'collect'

In [32]:
hashtags

In [34]:
full_tweet

NameError: name 'full_tweet' is not defined

In [35]:
table_df

In [38]:
words.saveAsTextFiles("out.txt")

Py4JJavaError: An error occurred while calling None.org.apache.spark.streaming.api.python.PythonTransformedDStream.
: java.lang.IllegalStateException: Adding new inputs, transformations, and output operations after starting a context is not supported
	at org.apache.spark.streaming.dstream.DStream.validateAtInit(DStream.scala:224)
	at org.apache.spark.streaming.dstream.DStream.<init>(DStream.scala:66)
	at org.apache.spark.streaming.api.python.PythonDStream.<init>(PythonDStream.scala:224)
	at org.apache.spark.streaming.api.python.PythonTransformedDStream.<init>(PythonDStream.scala:241)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.lang.reflect.Constructor.newInstance(Constructor.java:423)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)


In [40]:
words.shape()

AttributeError: 'TransformedDStream' object has no attribute 'shape'

In [41]:
words.map(lambda word: (word, 1)).reduceByKey(add)

AttributeError: 'NoneType' object has no attribute 'sc'

In [44]:
sqlContext.sql(
    "SELECT words FROM table_df LIMIT 10").collect().foreach(println)

AnalysisException: 'Table or view not found: table_df; line 1 pos 18'